# 03_builder

This notebook demonstrates how to use the **Builder** module in the `pysubmit` package.

The builder pattern provides a clean, modular way to inject logic into the simulation setup process.
It gives users hooks to modify or interact with the AEDT design in a structured and reusable way.

In this tutorial, we cover three different builder classes:
1. `DesignVariableBuilder`: Modify design parameters
2. `FunctionBuilder`: Run custom Python functions on the design
3. `ScriptBuilder`: Load and run user-defined Python functions stored in external `.py` files

### How Builders Fit into Larger Workflows
- **DesignVariableBuilder**, **FunctionBuilder**, and **ScriptBuilder** help you parameterize and automate repeated tasks (e.g. setting design variables, running custom logic, or loading external scripts).
- In a broader workflow, these builders can be combined with **Sweepers** (to vary parameters) and **Simulation** classes (to run HFSS analyses).
- This modular approach simplifies complex simulation pipelines.

> **Note:**
> These builders are especially useful when writing parameter sweeps or automating simulation workflows — topics also explored in other tutorials.


In [22]:
# opening the HFSS for demostration
from ansys.aedt.core.hfss import Hfss
hfss_file_path = './designs/simple_design.aedt'

hfss = Hfss(
    project=hfss_file_path,
    design='my_design',
    remove_lock=True,
    close_on_exit=True
)

PyAEDT INFO: Parsing ./designs/simple_design.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.13.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: File ./designs/simple_design.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_3c49ff27-023f-4441-ae6b-082fb4adb9e1.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 62005
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 18000.
PyAEDT INFO: Project simple_design has been opened.
PyAEDT INFO: Aedt Objects correctly read


## 1. Design Variable Builder

This builder is used to define and apply design variables in your AEDT simulation.
You can specify parameters as a dictionary, and the builder will apply them to the design automatically.

In [23]:
from pysubmit.workflow.builder import DesignVariableBuilder

# Create a builder
builder = DesignVariableBuilder(
    setup_name='Setup1',
    design_name='my_design'
)

We define a dictionary of parameters to apply to the design.

In [24]:
parameters = {'chip_base_length': '20mm'}

# Before and after comparison
print("CHIP BASE VALUE BEFORE:", hfss["chip_base_length"])

builded_parameters = builder.build(hfss, parameters=parameters)

print("CHIP BASE VALUE AFTER:", hfss["chip_base_length"])
# The PyAEDT logs may appear here; they can be ignored unless debugging.


CHIP BASE VALUE BEFORE: 16mm
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.13.0.
PyAEDT INFO: Returning found Desktop session with PID 18000!
PyAEDT INFO: Project simple_design set to active.
PyAEDT INFO: Aedt Objects correctly read
CHIP BASE VALUE AFTER: 20mm


## 2. Function Builder

The `FunctionBuilder` lets you define a custom function that receives the design object  
and performs arbitrary actions on it—ideal for scripting your own automation logic.

In [25]:
from pysubmit.workflow.builder import FunctionBuilder

def my_custom_function(hfss, value):
    hfss['chip_base_length'] = value
    return {'chip_base_length': value}

function_builder = FunctionBuilder(
    function=my_custom_function
)

We now use the function builder to execute the custom function within the HFSS design context.

In [26]:
print("CHIP BASE VALUE BEFORE:", hfss["chip_base_length"])

builded_parameters = function_builder.build(hfss, parameters={'value': '30mm'})

print("CHIP BASE VALUE AFTER:", hfss["chip_base_length"])

CHIP BASE VALUE BEFORE: 20mm
CHIP BASE VALUE AFTER: 30mm


## 3. Script Builder

The `ScriptBuilder` is used when your simulation logic is stored in a separate Python file.

Instead of importing your function manually into the notebook, you can simply provide the file path.  
`ScriptBuilder` will import and execute the function from that file in the context of the current HFSS design.

This is particularly helpful for modular projects where simulation setup logic is kept in external scripts.

In [27]:
from pysubmit.workflow.builder.script_builder import ScriptBuilder

script_builder = ScriptBuilder(
    path='./builder_scripts/script_builder_function.py'
)

print("CHIP BASE VALUE BEFORE:", hfss["chip_base_length"])
builded_parameters = script_builder.build(hfss, parameters={'value': '20mm'})
print("CHIP BASE VALUE AFTER:", hfss["chip_base_length"])

CHIP BASE VALUE BEFORE: 30mm
CHIP BASE VALUE AFTER: 20mm


In [28]:
hfss.release_desktop()


PyAEDT INFO: Desktop has been released and closed.


True